In [ ]:
# import sys
# sys.path.append(r"C:\Users\user\anaconda3\lib\site-packages\matlabengine-24.2-py3.12.egg")

# import matlab.engine
# eng = matlab.engine.start_matlab()
# print("MATLAB 연결 성공!")


In [ ]:
!pip install "stable-baselines3[extra]" gymnasium

In [ ]:
# 기존 주석 유지, 내부 로직만 Python으로 대체
import numpy as np
from gymnasium import Env
from gymnasium.spaces import Box
from stable_baselines3 import PPO
from stable_baselines3.common.env_checker import check_env
import math

# ✨ MATLAB 없이 동작하는 시뮬레이터 함수
def run_uav_sim(Kp, Ki, Kd, target=1.0, sim_time=5.0, dt=0.01):
    a = 1.0  # 시스템 시간 상수
    y = 0.0
    u = 0.0
    e_int = 0.0
    e_prev = 0.0
    ISE = 0.0

    for _ in range(int(sim_time / dt)):
        e = target - y
        e_int += e * dt
        e_der = (e - e_prev) / dt
        u = Kp * e + Ki * e_int + Kd * e_der

        y_dot = -a * y + u
        y += y_dot * dt

        ISE += e**2 * dt
        e_prev = e

    return -ISE  # ISE 기반 보상 반환


# 1. 강화학습 환경 정의
class PIDEnv(Env):
    def __init__(self):
        super().__init__()
        self.action_space = Box(low=np.array([0.1, 0.01, 0.01], dtype=np.float32),
                                high=np.array([5.0, 2.0, 2.0], dtype=np.float32),
                                dtype=np.float32)
        self.observation_space = Box(low=-np.inf, high=np.inf, shape=(1,), dtype=np.float32)
        self.state = np.array([0.0], dtype=np.float32)

        print("🔄 MATLAB 엔진 시작 중...")
        print("✅ MATLAB 엔진 시작 완료")

        # run_uav_sim.m과 state_derivative.m이 동일한 경로에 있어야 함
        # self.eng.cd(...) 제거

    def step(self, action):
        Kp, Ki, Kd = float(action[0]), float(action[1]), float(action[2])
        try:
            reward = float(run_uav_sim(Kp, Ki, Kd))
            if math.isnan(reward) or math.isinf(reward):
                print(f"[❌ NaN 보상] Kp={Kp:.3f}, Ki={Ki:.3f}, Kd={Kd:.3f}")
                reward = -1e6
        except Exception as e:
            print(f"[Python 시뮬 오류 발생]: {e}")
            reward = -1e6  # 심각한 오류 발생 시 큰 페널티

        terminated = True
        truncated = False
        return self.state, reward, terminated, truncated, {}

    def reset(self, *, seed=None, options=None):
        super().reset(seed=seed)
        self.state = np.array([0.0], dtype=np.float32)
        return self.state, {}

    def render(self, mode='human'):
        pass

    def close(self):
        pass  # self.eng.quit() 제거

# 2. 환경 생성 및 확인
env = PIDEnv()
check_env(env, warn=True)

# 3. PPO 에이전트 생성 및 학습
model = PPO("MlpPolicy", env, verbose=1)
model.learn(total_timesteps=30000)

# 4. 학습된 최적 PID 게인 확인
obs, _ = env.reset()
Kp, Ki, Kd = model.predict(obs, deterministic=True)[0]
print(f"🔧 최적 튜닝 결과: Kp = {Kp:.3f}, Ki = {Ki:.3f}, Kd = {Kd:.3f}")
